In [162]:
from src.heuristics import TicTacToeHeuristic
from src import TicTacToe
import time 

game_cls = TicTacToe()
heuristic = TicTacToeHeuristic()


def min_max(game: TicTacToe, is_max_player, cloned_game, depth):
    # start_time = time.time()
    get_result_start_time = time.time()
    result = game.is_terminated()
    get_result_end_time = time.time()
    # print("get result时间： " + str(get_result_end_time - get_result_start_time))
    if result and depth <= 3:
        get_winner_start = time.time()
        winner = game.get_winner()
        if winner == cloned_game.player:
            get_winner_end = time.time()
            print("get winner时间： " + str(get_winner_end - get_winner_start))
            return 10
        elif winner == 0:
            get_winner_end = time.time()
            print("get winner时间： " + str(get_winner_end - get_winner_start))
            return 0
        else:
            get_winner_end = time.time()
            print("get winner时间： " + str(get_winner_end - get_winner_start))
            return -10
    if depth > 3:
        h_score_start = time.time()
        h_score = heuristic(game.get_state(), cloned_game.player)
        h_score_end = time.time()
        print("计算h_score: " + str(h_score_end - h_score_start))
        return h_score
    # get_score_start_time = time.time()
    best_score = -float('inf') if is_max_player else float('inf')
    # get_score_end_time = time.time()
    # print("get score时间： " + str(get_score_end_time - get_score_start_time))
    for action in game.get_actions():
        # clone_start_time = time.time()
        new_game = game.clone()
        # clone_end_time = time.time()
        # print("clone执行时间：" + str(clone_end_time - clone_start_time))
        # move_start_time = time.time()
        new_game.move(action)
        # move_end_time = time.time()
        # print("move执行时间： " + str(move_end_time - move_start_time))
        score = min_max(new_game, not is_max_player, cloned_game, depth+1)
        if (is_max_player and score > best_score) or (not is_max_player and score < best_score):
            best_score = score
    # end_time = time.time()
    # print("min_max 执行时间：" + str(end_time - start_time))
    return best_score
    

def get_all_Qs(state: tuple[int, ...], player: int, action_space: set[int]) -> dict[int, float]:
    q_values = {}
    for action in action_space:

        new_game = game_cls.from_state(state, player)
        cloned_game = new_game.clone()
        new_game.move(action)
        score = min_max(new_game, False, cloned_game, depth=0)
        q_values[action] = score
    


get_all_Qs(game_cls.get_state(), game_cls.player, game_cls.get_actions())
print(get_all_Qs(game_cls.get_state(), game_cls.player, game_cls.get_actions()))

计算h_score: 0.0006268024444580078
计算h_score: 0.00018095970153808594
计算h_score: 0.00021386146545410156
计算h_score: 0.0002028942108154297
计算h_score: 0.00036907196044921875
计算h_score: 0.00014019012451171875
计算h_score: 0.00015306472778320312
计算h_score: 0.00013303756713867188
计算h_score: 0.000125885009765625
计算h_score: 0.00019407272338867188
计算h_score: 7.486343383789062e-05
计算h_score: 6.914138793945312e-05
计算h_score: 6.818771362304688e-05
计算h_score: 7.009506225585938e-05
计算h_score: 6.604194641113281e-05
计算h_score: 6.890296936035156e-05
计算h_score: 6.794929504394531e-05
计算h_score: 6.723403930664062e-05
计算h_score: 6.818771362304688e-05
计算h_score: 6.699562072753906e-05
计算h_score: 6.67572021484375e-05
计算h_score: 6.699562072753906e-05
计算h_score: 6.604194641113281e-05
计算h_score: 0.00011396408081054688
计算h_score: 7.295608520507812e-05
计算h_score: 6.985664367675781e-05
计算h_score: 6.914138793945312e-05
计算h_score: 6.914138793945312e-05
计算h_score: 6.794929504394531e-05
计算h_score: 6.794929504394531e-05
计算h_

In [ ]:
class AdversarialSearchAgent:
    def __init__(self, initial_state):
        self.initial_state = initial_state

    def _max_utility_actions(self, state, player, depth=None):
        if state.winner is not None:
            return state.utility(player), set()
        if depth is not None and depth == 0:
            return state.eval(player), set()
        utility_map = dict()
        best_set = set()
        max_utility = None
        for action in state.available_actions:
            result = state.copy_and_choose(action)
            next_depth = None if depth is None else depth - 1
            min_utility, _ = self._min_utility_actions(result, player, next_depth)
            utility_map[action] = min_utility
            print(utility_map)
            if max_utility is None:
                max_utility = min_utility
            elif max_utility < min_utility:
                max_utility = min_utility
        for action, utility in utility_map.items():
            if utility == max_utility:
                best_set.add(action)
        return max_utility, best_set

    def _min_utility_actions(self, state, player, depth=None):
        if state.winner is not None:
            return state.utility(player), set()
        if depth is not None and depth == 0:
            return state.eval(), set()
        utility_map = dict()
        best_set = set()
        min_utility = None
        for action in state.available_actions:
            result = state.copy_and_choose(action)
            next_depth = None if depth is None else depth - 1
            max_utility, _ = self._max_utility_actions(result, player, next_depth)
            utility_map[action] = max_utility
            if min_utility is None:
                min_utility = max_utility
            elif min_utility > max_utility:
                min_utility = max_utility
        for action, utility in utility_map.items():
            if utility == min_utility:
                best_set.add(action)
        return min_utility, best_set

    def h_minimax_search(self):
        return self._max_utility_actions(self.initial_state,
                                          self.initial_state.player, 4)

    def get_all_Qs(self):
        value_distribution = dict()
        start_time = time.time()
        for action in self.initial_state.available_actions:
            sub_initial_state = self.initial_state.copy_and_choose(action)
            value_distribution[action] = self._min_utility_actions(
                sub_initial_state,
                self.initial_state.player, 3
            )[0]
            end_time = time.time()
            print(action, end_time - start_time)
        return value_distribution

class TicTacToe:
    def __init__(self):
        self.player = 1
        self.board = [0] * 9
        self.history = []

    def __str__(self):
        temp_list = [*range(1, 10)]
        if self.board != [0] * 9:
            for i, player in enumerate(self.board):
                if player == 0:
                    temp_list[i] = ' '
                if player == 1:
                    temp_list[i] = 'X'
                if player == -1:
                    temp_list[i] = 'O'
        format_str = '\n+---+---+---+\n|{:^3}|{:^3}|{:^3}|' * 3 + '\n+---+---+---+\n'
        return format_str.format(*temp_list)

    def __eq__(self, other):
        return self.board == other.board

    # iterate through all possible lines
    def __iter__(self):
        for i in (0, 1, 2):
            yield self.board[i], self.board[i + 3], self.board[i + 6]
        for i in (0, 3, 6):
            yield self.board[i], self.board[i + 1], self.board[i + 2]
        for i in (0,):
            yield self.board[i], self.board[i + 4], self.board[i + 8]
        for i in (2,):
            yield self.board[i], self.board[i + 2], self.board[i + 4]


    # return all available actions under current state
    @property
    def available_actions(self):
        actions = set()
        for action in range(1, 10):
            if self.winner is None and self.board[action - 1] == 0:
                actions.add(action)
        return actions

    # return winner, 0 means tie. If there is no winner yet, return None.
    @property
    def winner(self):
        for first, second, third in self:
            if first == second == third != 0:
                return first
        for i in self.board:
            if i == 0:
                return None
        return 0

    # choose a position
    def choose(self, action):
        if action in self.available_actions:
            self.board[action - 1] = self.player
            self.history += [(self.player, action)]
            self.player = -self.player
        else:
            raise ValueError('Invalid position')

    def copy_and_choose(self, action):
        self_copy = copy.deepcopy(self)
        self_copy.choose(action)
        return self_copy
    
    def utility(self, player):
        return self.winner if self.winner == 0 or self.winner is None else 10 if self.winner == player else -10
    
    def eval(self, player):
        p1 = p2 = a1 = a2 = 0
        for line in self:
            p_count = line.count(player)
            a_count = line.count(-player)
            if p_count == 2 and a_count == 0:
                p2 += 1
            if p_count == 1 and a_count == 0:
                p1 += 1
            if p_count == 0 and a_count == 2:
                a2 += 1
            if p_count == 0 and a_count == 1:
                a1 += 1
        return 3 * p2 + p1 - (3 * a2 + a1)

    def start(self):
        print(self)
        player = 1
        while self.winner is None:
            print(f"Computer {player}'s turn: ")
            agent = AdversarialSearchAgent(copy.deepcopy(self))
            q_values = agent.get_all_Qs()
            print(q_values)
            self.choose(max(q_values, key=q_values.__getitem__))
            print(self)
            player = -player



class TicTacToeNew(object):

    @classmethod
    def from_state(cls, state: tuple[int, ...], player: int):
        obj = TicTacToe()
        obj.board = np.array(state)
        obj.player = player
        return obj

    @classmethod
    def eval(cls, state, player):
        pass

    def __init__(self, start_player=1, default_state_formatter: Callable[[tuple[int, ...]], str] = str):
        self.board = np.zeros(9)
        self.player = start_player
        self.default_state_formatter = default_state_formatter

    def move(self, action):
        if self.board[action] != 0:
            raise Exception("Invalid move")
        self.board[action] = self.player
        self.player = -self.player

    def agent_move(self, policy):
        best_action = policy(self.get_state(), self.player, self.get_actions())
        self.move(best_action)
        return best_action

    def reset(self, start_player=1):
        self.board = np.zeros(9)
        self.player = start_player

    def render(self, state_formatter: Optional[Callable[[tuple[int, ...]], str]] = None):
        formatter = state_formatter or self.default_state_formatter
        print(formatter(self.get_state()), flush=True)

    def get_state(self) -> tuple[int, ...]:
        return tuple(self.board.astype(int).tolist())

    def get_actions(self):
        return set(np.where(self.board == 0)[0].tolist())

    def get_winner(self):
        board = self.board.reshape([3,3])

        for i in range(3):
            if abs(board[i].sum()) == 3:
                return board[i][0]
            if abs(board[:, i].sum()) == 3:
                return board[0][i]

        for j in range(3):
            if abs(board.diagonal().sum()) == 3:
                return board[0, 0]

            if abs(np.fliplr(board).diagonal().sum()) == 3:
                return int(board[0, 2])
        return 0

    def is_terminated(self):
        return not self.get_actions() or self.get_winner()

    def update_state(self, state, player):
        self.board = np.array(state, dtype=int)
        self.player = player

    def clone(self):
        return deepcopy(self)

    def last_player(self):
        return -self.player


In [ ]:
game = TicTacToe()

In [ ]:
game.start()

Game started!

+---+---+---+
| 1 | 2 | 3 |
+---+---+---+
| 4 | 5 | 6 |
+---+---+---+
| 7 | 8 | 9 |
+---+---+---+

Computer 1's turn: 

Minimax evaluations:

+---+---+---+
|-2 |-3 |-2 |
+---+---+---+
|-3 | 0 |-3 |
+---+---+---+
|-2 |-3 |-2 |
+---+---+---+


+---+---+---+
|   |   |   |
+---+---+---+
|   | X |   |
+---+---+---+
|   |   |   |
+---+---+---+

Computer 2's turn: 

Minimax evaluations:

+---+---+---+
|-5 |-7 |-5 |
+---+---+---+
|-7 |   |-7 |
+---+---+---+
|-5 |-7 |-5 |
+---+---+---+


+---+---+---+
| O |   |   |
+---+---+---+
|   | X |   |
+---+---+---+
|   |   |   |
+---+---+---+

Computer 1's turn: 

+---+---+---+
| O | X |   |
+---+---+---+
|   | X |   |
+---+---+---+
|   |   |   |
+---+---+---+

Computer 2's turn: 

+---+---+---+
| O | X |   |
+---+---+---+
|   | X |   |
+---+---+---+
|   | O |   |
+---+---+---+

Computer 1's turn: 

+---+---+---+
| O | X |   |
+---+---+---+
| X | X |   |
+---+---+---+
|   | O |   |
+---+---+---+

Computer 2's turn: 

+---+---+---+
| O | X

In [71]:
import numpy as np
from copy import deepcopy
from typing import Callable, Optional


class TicTacToe():

    @classmethod
    def from_state(cls, state: tuple[int, ...], player: int):
        obj = TicTacToe()
        obj.board = np.array(state)
        obj.player = player
        return obj

    @classmethod
    def eval(cls, state, player):
        pass

    def __init__(self, start_player=1, default_state_formatter: Callable[[tuple[int, ...]], str] = str):
        self.board = np.zeros(9)
        self.player = start_player
        self.default_state_formatter = default_state_formatter
        self.action_history = []

    def move(self, action):
        if self.board[action] != 0:
            raise Exception("Invalid move")
        self.board[action] = self.player
        self.player = -self.player
        self.action_history.append(action)

    def agent_move(self, policy):
        best_action = policy(self.get_state(), self.player, self.get_actions())
        self.move(best_action)
        return best_action

    def reset(self, start_player=1):
        self.board = np.zeros(9)
        self.player = start_player

    def render(self, state_formatter: Optional[Callable[[tuple[int, ...]], str]] = None):
        formatter = state_formatter or self.default_state_formatter
        print(formatter(self.get_state()), flush=True)

    def get_state(self) -> tuple[int, ...]:
        return tuple(self.board.astype(int).tolist())

    def get_actions(self):
        return set(np.where(self.board == 0)[0].tolist())

    def get_winner(self):
        board = self.board.reshape([3,3])

        for i in range(3):
            if abs(board[i].sum()) == 3:
                return board[i][0]
            if abs(board[:, i].sum()) == 3:
                return board[0][i]

        for j in range(3):
            if abs(board.diagonal().sum()) == 3:
                return board[0, 0]

            if abs(np.fliplr(board).diagonal().sum()) == 3:
                return int(board[0, 2])
        return 0

    def is_terminated(self):
        return not self.get_actions() or self.get_winner()

    def update_state(self, state, player):
        self.board = np.array(state, dtype=int)
        self.player = player

    def clone(self):
        return deepcopy(self)

    def last_player(self):
        return -self.player

    def apply_action(self, action):
        if self.board[action] != 0:
            raise Exception("Invalid move")
        self.board[action] = self.player
        self.player = -self.player
        self.action_history.append(action)

    def undo_action(self):
        if not self.action_history:
            raise Exception("No actions to undo")
        last_action = self.action_history.pop()
        self.board[last_action] = 0
        self.player = -self.player
        

In [127]:
from src.heuristics import TicTacToeHeuristic
import time

game_cls = TicTacToe()
heuristic = TicTacToeHeuristic()

def min_max(game: TicTacToe, is_max_player, cloned_game, depth):
    if game.is_terminated():
        winner = game.get_winner()
        if winner == cloned_game.player:
            return 10
        elif winner == 0:
            return 0
        else:
            return -10
    if depth == 0:
        return heuristic(game.get_state(), cloned_game.player)

    best_score = -float('inf') if is_max_player else float('inf')
    for action in game.get_actions():
        game.apply_action(action)
        score = min_max(game, not is_max_player, cloned_game, depth - 1)
        game.undo_action()
        if (is_max_player and score > best_score) or (not is_max_player and score < best_score):
            best_score = score
    return best_score


def get_all_Qs(state: tuple[int, ...], player: int, action_space: set[int]) -> dict[int, float]:
    q_values = {}
    for action in action_space:
        new_game = game_cls.from_state(state, player)
        cloned_game = new_game.clone()
        new_game.move(action)
        score = min_max(new_game, False, cloned_game, depth=3)
        q_values[action] = score
    return q_values


In [128]:
game_cls.reset()
game_cls.move(4)
game_cls.render()

In [129]:
result = get_all_Qs(game_cls.get_state(), game_cls.player, game_cls.get_actions())
print(result)

In [130]:
game_cls.move(0)
game_cls.render()

In [131]:
game_cls.move(1)
game_cls.render()

In [132]:
result = get_all_Qs(game_cls.get_state(), game_cls.player, game_cls.get_actions())
print(result)

In [133]:
game_cls.move(7)
game_cls.render()

In [134]:
game_cls.move(2)
game_cls.render()

In [135]:
result = get_all_Qs(game_cls.get_state(), game_cls.player, game_cls.get_actions())
print(result)